In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType
import pyspark.sql.functions as f

In [93]:
spark = SparkSession.builder \
    .appName("improve-enrich-data") \
    .getOrCreate()

In [94]:
# Definir o schema dos dados na leitura
schema = StructType([
    StructField("invoice_no", StringType(), True),
    StructField("invoice_date", StringType(), True), # observe que é um tipo string para uma coluna de datetime ;)
    StructField("customer_id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("category", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("selling_price_per_unit", DoubleType(), True),
    StructField("cost_price_per_unit", DoubleType(), True),
    StructField("payment_method", StringType(), True),
    StructField("region", StringType(), True),
    StructField("state", StringType(), True),
    StructField("shopping_mall", StringType(), True)
])

In [95]:
'''
Sobre o Conjunto de Dados
Visão Geral

Este conjunto de dados é uma criação fictícia, projetada especificamente para ajudar analistas de 
dados a praticar análise exploratória de dados e visualização de dados. 
Ele contém informações sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de 
supermercado.
'''
df = spark.read.csv("datasource/Different_stores_dataset.csv", header=True, schema=schema)

In [96]:
df.show(n=3, truncate=False, vertical=True)

-RECORD 0---------------------------------
 invoice_no             | I138884         
 invoice_date           | 10/30/2016 9:58 
 customer_id            | C241288         
 gender                 | Female          
 age                    | 28              
 category               | Clothing        
 quantity               | 5               
 selling_price_per_unit | 1500.4          
 cost_price_per_unit    | 1425.38         
 payment_method         | Credit Card     
 region                 | South           
 state                  | Kentucky        
 shopping_mall          | Kanyon          
-RECORD 1---------------------------------
 invoice_no             | I317333         
 invoice_date           | 10/30/2016 9:58 
 customer_id            | C111565         
 gender                 | Male            
 age                    | 21              
 category               | Shoes           
 quantity               | 3               
 selling_price_per_unit | 1800.51         
 cost_price

In [97]:
from ydata_profiling import ProfileReport
'''
Esta linha importa a classe ProfileReport do módulo ydata_profiling que é uma biblioteca para análise exploratória de dados. 
Seu objetivo é gerar relatórios de perfil de conjuntos de dados, oferecendo uma visão geral abrangente dos dados.
'''
report = ProfileReport(
    df,
    title='Different stores dataset',       
    dataset={
        "description": "Dados fictícios sobre pedidos feitos por clientes em um aplicativo de entrega de produtos de supermercado",
        "copyright_holder": "Kushal Kamra",
        "url": "https://www.kaggle.com/datasets/kzmontage/sales-from-different-stores",
    },
    variables={
        "descriptions": {
            "invoice_no": "Número da fatura associado a cada transação.",
            "customer_id": "Identificador único para cada cliente.",
            "gender": "Gênero do cliente (presumido ser binário: masculino/feminino).",
            "age": "Idade do cliente.",
            "category": "Categoria do produto associada à transação.",
            "quantity": "Quantidade de produtos comprados em cada transação.",
            "selling_price_per_unit": "Preço de venda por unidade do produto.",
            "cost_price_per_unit": "Preço de custo por unidade do produto.",
            "payment_method": "Método de pagamento usado na transação.",
            "region": "Região geográfica associada à transação.",
            "state": "Estado onde a transação ocorreu.",
            "shopping_mall": "Shopping center onde ocorreu a transação.",
        }
    },
    infer_dtypes=False,
    interactions=None,
    missing_diagrams=None,
    correlations={"pearson": {"calculate": True}}
)

In [ ]:
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

In [82]:
# Limpeza e padronização dos dados: Vamos melhorar o schema dos dados modificando o _Tipo_ da coluna `invoice_date` de String para Timestamp. 
df = df.withColumn("invoice_datetime", f.to_timestamp(df.invoice_date, 'MM/dd/yyyy H:mm'))


In [83]:
# Padronização de Dados: Criar uma coluna extra invoice_date do _Tipo_ String no formato 'dd/MM/yyyy' para contrução de partição 
# melhorando o desempenho de consultas com data.
df = df.withColumn("invoice_date", f.date_format("invoice_datetime", 'dd/MM/yyyy'))

In [84]:
df.select("invoice_datetime", "invoice_date").show()

+-------------------+------------+
|   invoice_datetime|invoice_date|
+-------------------+------------+
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:58:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 09:59:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:00:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:01:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
|2016-10-30 10:02:00|  30/10/2016|
+-------------------+------------+
only showing top 20 rows



In [85]:
df.printSchema()

root
 |-- invoice_no: string (nullable = true)
 |-- invoice_date: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- selling_price_per_unit: double (nullable = true)
 |-- cost_price_per_unit: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- region: string (nullable = true)
 |-- state: string (nullable = true)
 |-- shopping_mall: string (nullable = true)
 |-- invoice_datetime: timestamp (nullable = true)



In [86]:
df.createOrReplaceTempView("stores")

In [88]:
spark.sql(
    "SELECT * FROM stores"
).show(n=1, vertical=True)

-RECORD 0-------------------------------------
 invoice_no             | I138884             
 invoice_date           | 30/10/2016          
 customer_id            | C241288             
 gender                 | Female              
 age                    | 28                  
 category               | Clothing            
 quantity               | 5                   
 selling_price_per_unit | 1500.4              
 cost_price_per_unit    | 1425.38             
 payment_method         | Credit Card         
 region                 | South               
 state                  | Kentucky            
 shopping_mall          | Kanyon              
 invoice_datetime       | 2016-10-30 09:58:00 
only showing top 1 row



In [89]:
'''
1. Melhoramento e Enriquecimento de Dados:
    Você poderia enriquecer esses dados adicionando informações de rentabilidade para cada 
    transação (calculando a diferença entre selling_price_per_unit e cost_price_per_unit
    multiplicada pela quantity).
'''
# Comandos Renata Soares
df = df.withColumn("rentabilidade",(df.selling_price_per_unit- df.cost_price_per_unit) * df.quantity ) 
df.printSchema()
df.createOrReplaceTempView("stores")
spark.sql(
    "SELECT * FROM stores"
).show(n=1, vertical=True)



root
 |-- invoice_no: string (nullable = true)
 |-- invoice_date: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- selling_price_per_unit: double (nullable = true)
 |-- cost_price_per_unit: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- region: string (nullable = true)
 |-- state: string (nullable = true)
 |-- shopping_mall: string (nullable = true)
 |-- invoice_datetime: timestamp (nullable = true)
 |-- rentabilidade: double (nullable = true)

-RECORD 0-------------------------------------
 invoice_no             | I138884             
 invoice_date           | 30/10/2016          
 customer_id            | C241288             
 gender                 | Female              
 age                    | 28                  
 category               | Clothing            
 quantity     

In [90]:
''' 
2. Eliminação de Dados Irrelevantes:
    Eliminar dados irrelevantes significa remover informações que não contribuem para a análise ou 
    objetivo específico.

    Se o objetivo for analisar apenas as vendas de produtos, campos como invoice_no e customer_id 
    podem ser considerados irrelevantes e, portanto, removidos.
    Em análises focadas em tendências, detalhes como shopping_mall podem ser descartados.
'''
# Comandos Renata Soares
# removendo a coluna payment_method pois invoice_no, customer_id e shopping_mall são usandos em outros exercícios
df = df.drop('payment_method')
df.show(n=3, vertical=True)

-RECORD 0-------------------------------------
 invoice_no             | I138884             
 invoice_date           | 30/10/2016          
 customer_id            | C241288             
 gender                 | Female              
 age                    | 28                  
 category               | Clothing            
 quantity               | 5                   
 selling_price_per_unit | 1500.4              
 cost_price_per_unit    | 1425.38             
 payment_method         | Credit Card         
 region                 | South               
 state                  | Kentucky            
 invoice_datetime       | 2016-10-30 09:58:00 
 rentabilidade          | 375.0999999999999   
-RECORD 1-------------------------------------
 invoice_no             | I317333             
 invoice_date           | 30/10/2016          
 customer_id            | C111565             
 gender                 | Male                
 age                    | 21                  
 category    

In [68]:
'''
3. Granulação e Agregação:
    A granulação e agregação envolvem resumir ou combinar dados para um nível de detalhe mais 
    alto ou mais baixo, dependendo da necessidade da análise.

    3.1 Granulação: Filtrar dados para um nível mais detalhado, como todas as transações de um 
    cliente específico (customer_id).

    3.2 Agregação: Sumarizar dados, como calcular o total de vendas por category ou a média de 
    idade dos clientes (age) por gender.
'''

#Comandos Renata Soares 3.1 - Granulação filtro por customer_id
#spark.sql(
#    "SELECT * FROM stores WHERE customer_id = 'C266599'"
#).show(vertical=True)

#Comandos Renata Soares 3.2 - Agregação - Total de Vendas por categoria
spark.sql(
    "SELECT category, sum (selling_price_per_unit*selling_price_per_unit) AS total_vendas FROM stores GROUP BY category"
).show(n=4,vertical=True)

-RECORD 0----------------------------
 category     | Food & Beverage      
 total_vendas | 4443068.311500132    
-RECORD 1----------------------------
 category     | Clothing             
 total_vendas | 3.420815705528203E10 
-RECORD 2----------------------------
 category     | Souvenir             
 total_vendas | 7458223.144499951    
-RECORD 3----------------------------
 category     | Books                
 total_vendas | 1.2643476434999892E7 
only showing top 4 rows



In [91]:
'''
4. Consistência, Integridade, Concordância:
    Esses conceitos referem-se a garantir que os dados sejam precisos, confiáveis e 
    consistentes em todo o conjunto de dados.

    Verificar se todos os invoice_date estão no formato correto e se os valores de age 
    são razoáveis (por exemplo, não negativos ou excessivamente altos).
    Assegurar que selling_price_per_unit seja sempre maior que cost_price_per_unit, 
    pois o contrário indicaria uma inconsistência nos dados
'''

# Comandos Renata Soares

#Verificação Age - se existe idade negativa ou idade maior ou igual 100
#spark.sql(
#    "SELECT * FROM stores where age < 0 or age >= 100  "
#).show(vertical=True)

# Verificação selling_price_per_unit - se existe selling_price_per_unit < cost_price_per_unit
#spark.sql(
#   "SELECT * FROM stores where selling_price_per_unit < cost_price_per_unit   "
#).show(n=1, vertical=True)

# No dataframe caso exista selling_price_per_unit < cost_price_per_unit então selling_price_per_unit *2 senão selling_price_per_unit 
df = df.withColumn("selling_price_per_unit",f.when(df.selling_price_per_unit < df.cost_price_per_unit,df.selling_price_per_unit*2).otherwise(df.selling_price_per_unit)).show(vertical=True)



-RECORD 0-------------------------------------
 invoice_no             | I138884             
 invoice_date           | 30/10/2016          
 customer_id            | C241288             
 gender                 | Female              
 age                    | 28                  
 category               | Clothing            
 quantity               | 5                   
 selling_price_per_unit | 1500.4              
 cost_price_per_unit    | 1425.38             
 payment_method         | Credit Card         
 region                 | South               
 state                  | Kentucky            
 invoice_datetime       | 2016-10-30 09:58:00 
 rentabilidade          | 375.0999999999999   
-RECORD 1-------------------------------------
 invoice_no             | I317333             
 invoice_date           | 30/10/2016          
 customer_id            | C111565             
 gender                 | Male                
 age                    | 21                  
 category    

In [51]:
'''
5. Duplicação e Redundância:
    Dados duplicados ou redundantes podem distorcer análises e devem ser removidos ou tratados.
    
    Procurar por registros duplicados (por exemplo, duas transações com o mesmo invoice_no).
    Identificar redundâncias, como se a combinação de region e state sempre leva ao 
    mesmo shopping_mall, indicando que um desses campos pode ser redundante.
'''
# Comandos Renata Soares - Duplicação
#spark.sql(
#    "select invoice_no, count(*) from stores group by invoice_no having count(*) > 1  "
#).show(vertical=True)

# Comandos Renata Soares - Redundância

(0 rows)

